# Server

In [6]:
%matplotlib inline
import IPython.core.display         
# setup output image format (Chrome works best)
IPython.core.display.set_matplotlib_formats("svg")
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
from sklearn import *
from scipy import stats
random.seed(100)
import pandas as pd
import csv
import numpy as np
import os

In [7]:
# use TensorFlow backend
%env KERAS_BACKEND=tensorflow     
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Activation, Dropout, Conv2D, Flatten, \
                Input, GlobalAveragePooling2D
from keras.preprocessing import image

import keras
import tensorflow
import logging
logging.basicConfig()
import struct

# use channels first representation for images
from keras import backend as K
K.set_image_data_format('channels_first')

from keras.callbacks import TensorBoard

from joblib import dump, load
scaler = load('minmax_scaler.sav')

# load trained model from json
model_file = open('model.json', 'r')
model = model_file.read()
model_file.close()
nn = model_from_json(model)
nn.load_weights('model.h5')
print('Loaded model from files')

a = array([[1, 0, 0, 0, 9, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5]])
a  = scaler.transform(a)
predY = nn.predict_proba(a, verbose=False)

env: KERAS_BACKEND=tensorflow
Loaded model from files


In [ ]:
# import socket programming library 
import socket 
import json

# import thread module 
from _thread import *
import threading 

def send_thread(c, b): 
    c.send(b + b'\n')

def exec_recv_data(j):
    a = j['action']
    if a == 1:
        append_to_file(j, c)
    elif a == 2:
        predict_action(j, c)

def predict_action(j):
    print('Predict action by {}, length: {}'.format(j['input'], len(j['input'])) )    
    a  = scaler.transform(array([j['input']]))
    predY = nn.predict_proba(a, verbose=False)
    return list(predY[0])
    
def append_to_file(j):
    t = j['type']
    filename = 'texasholdem_type_{}.txt'.format(t)
    f = open(filename, 'a')
    f.write(j['data'] + '\n')
    f.close()
    print('Saved to {}'.format(filename))

print_lock = threading.Lock() 

# thread fuction 
def threaded(c):
    data_str = '' 
    while True: 
        try:
            # data received from client 
            data = c.recv(1024)
            print("recv: " + data.decode('utf-8'))

            data_str += data.decode('utf-8')
            for d in data_str.splitlines():
                try:
                    j = json.loads(d)
                    a = j['action']
                    if a == 1:
                        append_to_file(j)
                        c.send(json.dumps({'success': [j['uuid']]}).encode() + b'\n')
                    elif a == 2:
                        pred = predict_action(j)
                        c.send(b'{'+bytes(bytearray('"output": [{:f},{:f},{:f}]'.format(pred[0],pred[1],pred[2]), 'utf-8')) + b'}\n')
                    data_str = ''
                    
                except:
                    data_str = d
                    
            
            if not data: 
                print('Bye') 

                # lock released on exit 
                print_lock.release() 
                break

            # reverse the given string from client 
    #         data = data[::-1] 

            # send back reversed string to client 
#             c.send(data+b'\n') 
        except:
            print("error")
            print_lock.release()
            break
            
    # connection closed 
    c.close() 
  
  
host = "127.0.0.1" 
# reverse a port on your computer 
port = 8808

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
s.bind((host, port)) 
print("socket binded to post", port) 

# put the socket into listening mode 
s.listen(5) 
print("socket is listening") 

# a forever loop until client wants to exit 
while True: 

    # establish connection with client 
    c, addr = s.accept() 

    # lock acquired by client 
    print_lock.acquire() 
    print('Connected to :', addr[0], ':', addr[1]) 

    # Start a new thread and return its identifier 
    start_new_thread(threaded, (c,)) 
s.close() 


socket binded to post 8808
socket is listening
Connected to : 127.0.0.1 : 8257
recv: {"action":2,"input":[1,0,0,0,11,2,4,3,0,0,0,0,0,0,0,0,0,0,0.5]}

Predict action by [1, 0, 0, 0, 11, 2, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5], length: 19
recv: {"action":2,"input":[0,1,0,0,11,2,4,3,2,2,10,3,0,0,0,0,0,0,1]}

Predict action by [0, 1, 0, 0, 11, 2, 4, 3, 2, 2, 10, 3, 0, 0, 0, 0, 0, 0, 1], length: 19
recv: {"action":2,"input":[0,0,1,0,11,2,4,3,2,2,10,3,0,0,0,0,0,0,1]}

Predict action by [0, 0, 1, 0, 11, 2, 4, 3, 2, 2, 10, 3, 0, 0, 0, 0, 0, 0, 1], length: 19
recv: {"action":2,"input":[0,0,0,1,11,2,4,3,2,2,10,3,10,2,0,0,0,0,1]}

Predict action by [0, 0, 0, 1, 11, 2, 4, 3, 2, 2, 10, 3, 10, 2, 0, 0, 0, 0, 1], length: 19
recv: {"action":2,"input":[1,0,0,0,3,0,1,3,0,0,0,0,0,0,0,0,0,0,0.3333333333333333]}

Predict action by [1, 0, 0, 0, 3, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.3333333333333333], length: 19
recv: {"action":2,"input":[0,1,0,0,3,0,1,3,11,1,7,0,0,0,0,0,0,0,1]}

Predict action by 

recv: {"action":2,"input":[0,1,0,0,13,2,5,1,13,0,3,3,0,0,0,0,0,0,1]}

Predict action by [0, 1, 0, 0, 13, 2, 5, 1, 13, 0, 3, 3, 0, 0, 0, 0, 0, 0, 1], length: 19
recv: {"action":2,"input":[0,0,1,0,13,2,5,1,13,0,3,3,0,0,0,0,0,0,1]}

Predict action by [0, 0, 1, 0, 13, 2, 5, 1, 13, 0, 3, 3, 0, 0, 0, 0, 0, 0, 1], length: 19
recv: {"action":2,"input":[0,0,0,1,13,2,5,1,13,0,3,3,8,2,0,0,0,0,1]}

Predict action by [0, 0, 0, 1, 13, 2, 5, 1, 13, 0, 3, 3, 8, 2, 0, 0, 0, 0, 1], length: 19
recv: {"action":2,"input":[1,0,0,0,12,2,13,0,0,0,0,0,0,0,0,0,0,0,0.3333333333333333]}

Predict action by [1, 0, 0, 0, 12, 2, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.3333333333333333], length: 19
recv: {"action":2,"input":[0,1,0,0,12,2,13,0,11,0,6,0,0,0,0,0,0,0,1]}

Predict action by [0, 1, 0, 0, 12, 2, 13, 0, 11, 0, 6, 0, 0, 0, 0, 0, 0, 0, 1], length: 19
recv: {"action":2,"input":[0,0,0,1,12,2,13,0,11,0,6,0,3,1,0,0,0,0,1]}

Predict action by [0, 0, 0, 1, 12, 2, 13, 0, 11, 0, 6, 0, 3, 1, 0, 0, 0, 0, 1], length: 19
re

recv: {"action":2,"input":[0,0,1,0,8,1,8,0,11,1,7,1,0,0,0,0,0,0,1]}

Predict action by [0, 0, 1, 0, 8, 1, 8, 0, 11, 1, 7, 1, 0, 0, 0, 0, 0, 0, 1], length: 19
recv: {"action":2,"input":[0,0,0,1,8,1,8,0,11,1,7,1,12,3,0,0,0,0,1]}

Predict action by [0, 0, 0, 1, 8, 1, 8, 0, 11, 1, 7, 1, 12, 3, 0, 0, 0, 0, 1], length: 19
recv: {"action":2,"input":[1,0,0,0,3,0,2,3,0,0,0,0,0,0,0,0,0,0,0.3333333333333333]}

Predict action by [1, 0, 0, 0, 3, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.3333333333333333], length: 19
recv: {"action":2,"input":[0,1,0,0,3,0,2,3,13,3,1,0,0,0,0,0,0,0,1]}

Predict action by [0, 1, 0, 0, 3, 0, 2, 3, 13, 3, 1, 0, 0, 0, 0, 0, 0, 0, 1], length: 19
recv: {"action":2,"input":[0,0,0,1,3,0,2,3,13,3,1,0,7,3,0,0,0,0,1]}

Predict action by [0, 0, 0, 1, 3, 0, 2, 3, 13, 3, 1, 0, 7, 3, 0, 0, 0, 0, 1], length: 19
recv: {"action":2,"input":[1,0,0,0,10,3,2,1,0,0,0,0,0,0,0,0,0,0,0.5]}

Predict action by [1, 0, 0, 0, 10, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5], length: 19
recv: {"action

recv: {"action":2,"input":[0,0,0,1,12,1,13,2,11,2,12,2,5,1,0,0,0,0,1]}

Predict action by [0, 0, 0, 1, 12, 1, 13, 2, 11, 2, 12, 2, 5, 1, 0, 0, 0, 0, 1], length: 19
recv: {"action":2,"input":[1,0,0,0,10,0,1,2,0,0,0,0,0,0,0,0,0,0,0.3333333333333333]}

Predict action by [1, 0, 0, 0, 10, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.3333333333333333], length: 19
recv: {"action":2,"input":[0,1,0,0,10,0,1,2,10,1,2,1,0,0,0,0,0,0,1]}

Predict action by [0, 1, 0, 0, 10, 0, 1, 2, 10, 1, 2, 1, 0, 0, 0, 0, 0, 0, 1], length: 19
recv: {"action":2,"input":[0,0,0,1,10,0,1,2,10,1,2,1,1,3,0,0,0,0,1]}

Predict action by [0, 0, 0, 1, 10, 0, 1, 2, 10, 1, 2, 1, 1, 3, 0, 0, 0, 0, 1], length: 19
recv: {"action":2,"input":[1,0,0,0,12,1,7,0,0,0,0,0,0,0,0,0,0,0,0.5]}

Predict action by [1, 0, 0, 0, 12, 1, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5], length: 19
recv: {"action":2,"input":[0,1,0,0,12,1,7,0,3,3,12,3,0,0,0,0,0,0,1]}

Predict action by [0, 1, 0, 0, 12, 1, 7, 0, 3, 3, 12, 3, 0, 0, 0, 0, 0, 0, 1], length: 19
re

recv: {"action":2,"input":[0,1,0,0,12,1,12,2,8,1,7,0,0,0,0,0,0,0,0]}

Predict action by [0, 1, 0, 0, 12, 1, 12, 2, 8, 1, 7, 0, 0, 0, 0, 0, 0, 0, 0], length: 19
recv: 
Bye
Connected to : 127.0.0.1 : 9076
recv: {"action":2,"input":[1,0,0,0,3,0,1,3,0,0,0,0,0,0,0,0,0,0,0]}

Predict action by [1, 0, 0, 0, 3, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], length: 19
recv: 
Bye
Connected to : 127.0.0.1 : 9108
recv: {"action":2,"input":[1,0,0,0,6,2,6,1,0,0,0,0,0,0,0,0,0,0,0]}

Predict action by [1, 0, 0, 0, 6, 2, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], length: 19
recv: {"action":2,"input":[0,1,0,0,6,2,6,1,4,3,13,0,0,0,0,0,0,0,0]}

Predict action by [0, 1, 0, 0, 6, 2, 6, 1, 4, 3, 13, 0, 0, 0, 0, 0, 0, 0, 0], length: 19
recv: 
Bye
Connected to : 127.0.0.1 : 9138
recv: {"action":2,"input":[1,0,0,0,12,1,1,0,0,0,0,0,0,0,0,0,0,0,0]}

Predict action by [1, 0, 0, 0, 12, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], length: 19
recv: {"action":2,"input":[0,1,0,0,12,1,1,0,10,1,9,3,0,0,0,0,0,0,0]}

Predict actio

...

recv: {"action":2,"input":[1,0,0,0,7,2,1,1,0,0,0,0,0,0,0,0,0,0,0]}

Predict action by [1, 0, 0, 0, 7, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], length: 19
recv: {"action":2,"input":[1,0,0,0,7,2,1,1,0,0,0,0,0,0,0,0,0,0,0]}

Predict action by [1, 0, 0, 0, 7, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], length: 19
recv: {"action":2,"input":[1,0,0,0,7,2,1,1,0,0,0,0,0,0,0,0,0,0,0]}

Predict action by [1, 0, 0, 0, 7, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], length: 19
recv: {"action":2,"input":[1,0,0,0,7,2,1,1,0,0,0,0,0,0,0,0,0,0,0]}

Predict action by [1, 0, 0, 0, 7, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], length: 19
recv: {"action":2,"input":[0,1,0,0,7,2,1,1,2,3,11,0,0,0,0,0,0,0,0]}

Predict action by [0, 1, 0, 0, 7, 2, 1, 1, 2, 3, 11, 0, 0, 0, 0, 0, 0, 0, 0], length: 19
recv: {"action":2,"input":[0,1,0,0,7,2,1,1,2,3,11,0,0,0,0,0,0,0,0]}

Predict action by [0, 1, 0, 0, 7, 2, 1, 1, 2, 3, 11, 0, 0, 0, 0, 0, 0, 0, 0], length: 19
recv: {"action":2,"input":[0,0,1,0,7,2,1,1,2,3,11,0,0,0,0,0,

recv: {"action":2,"input":[0,0,1,0,1,0,13,3,5,3,12,0,0,0,0,0,0,0,0]}

Predict action by [0, 0, 1, 0, 1, 0, 13, 3, 5, 3, 12, 0, 0, 0, 0, 0, 0, 0, 0], length: 19
recv: {"action":2,"input":[0,0,0,1,1,0,13,3,5,3,12,0,3,3,0,0,0,0,0]}

Predict action by [0, 0, 0, 1, 1, 0, 13, 3, 5, 3, 12, 0, 3, 3, 0, 0, 0, 0, 0], length: 19
recv: {"action":2,"input":[0,0,0,1,1,0,13,3,5,3,12,0,3,3,0,0,0,0,0]}

Predict action by [0, 0, 0, 1, 1, 0, 13, 3, 5, 3, 12, 0, 3, 3, 0, 0, 0, 0, 0], length: 19
recv: {"action":2,"input":[1,0,0,0,7,1,11,1,0,0,0,0,0,0,0,0,0,0,0]}

Predict action by [1, 0, 0, 0, 7, 1, 11, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], length: 19
recv: {"action":2,"input":[1,0,0,0,7,1,11,1,0,0,0,0,0,0,0,0,0,0,0]}

Predict action by [1, 0, 0, 0, 7, 1, 11, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], length: 19
recv: {"action":2,"input":[1,0,0,0,7,1,11,1,0,0,0,0,0,0,0,0,0,0,0]}

Predict action by [1, 0, 0, 0, 7, 1, 11, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], length: 19
recv: {"action":2,"input":[1,0,0,0,7,1,11,1,0,

recv: {"action":2,"input":[0,0,1,0,4,0,2,3,13,0,11,3,0,0,0,0,0,0,0]}

Predict action by [0, 0, 1, 0, 4, 0, 2, 3, 13, 0, 11, 3, 0, 0, 0, 0, 0, 0, 0], length: 19
recv: {"action":2,"input":[0,0,0,1,4,0,2,3,13,0,11,3,7,2,0,0,0,0,0]}

Predict action by [0, 0, 0, 1, 4, 0, 2, 3, 13, 0, 11, 3, 7, 2, 0, 0, 0, 0, 0], length: 19
recv: {"action":2,"input":[0,0,0,1,4,0,2,3,13,0,11,3,7,2,0,0,0,0,0]}

Predict action by [0, 0, 0, 1, 4, 0, 2, 3, 13, 0, 11, 3, 7, 2, 0, 0, 0, 0, 0], length: 19
recv: 
Bye
Connected to : 127.0.0.1 : 9840
recv: {"action":2,"input":[1,0,0,0,7,0,1,0,0,0,0,0,0,0,0,0,0,0,0]}

Predict action by [1, 0, 0, 0, 7, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], length: 19
recv: {"action":2,"input":[0,1,0,0,7,0,1,0,8,3,6,1,0,0,0,0,0,0,0]}

Predict action by [0, 1, 0, 0, 7, 0, 1, 0, 8, 3, 6, 1, 0, 0, 0, 0, 0, 0, 0], length: 19
recv: {"action":2,"input":[0,1,0,0,7,0,1,0,8,3,6,1,0,0,0,0,0,0,0]}

Predict action by [0, 1, 0, 0, 7, 0, 1, 0, 8, 3, 6, 1, 0, 0, 0, 0, 0, 0, 0], length: 19
recv: {"a

Saved to texasholdem_type_3.txt
Saved to texasholdem_type_3.txt
Saved to texasholdem_type_3.txt
recv: d67274e9fb1"}
{"action":1,"data":"STATE:10:r200r300r700cr2300r3900c\/r4000c\/r4100c\/r4200c:4c6h|5d5c\/Jd7hAh\/Kd\/8d:-4200|4200:AIPlayer|Human","type":3,"uuid":"8731c192-cf61-4392-b20b-fb2ffbc40ad6"}
{"action":1,"data":"STATE:11:r200r300cr1100r1900r2700cc\/cr2800c\/cr2900c\/r3300r3700c:4h7d|4c4d\/8s6dTs\/Kd\/2c:-3700|3700:Human|AIPlayer","type":3,"uuid":"4e22ed18-c837-4728-b394-eb727e1acd5e"}
{"action":1,"data":"STATE:12:r500r900r1700cr8100r14500c\/r14600c\/r14700c\/r14800c:Kd9c|6c7s\/8c2h8s\/Th\/Jd:14800|-14800:AIPlayer|Human","type":3,"uuid":"87a41749-87dd-40e7-bde8-b6167c87039e"}
{"action":1,"data":"STATE:13:r200c\/cr300c\/cr400c\/cr500r1300cc:Kh2c|7s9s\/6s9hKc\/Th\/3s:1300|-1300:Human|AIPlayer","type":3,"uuid":"3c96f28b-a065-46eb-8b83-d7ca148b6731"}
{"action":1,"data":"STATE:14:r300r500r700cr1500r2300r3900cc\/r4000c\/r4100r4300cr4500r4700c\/r4800r4900cc:9s2s|Jc4d\/7c6dKc\/6s\/8d:-

Saved to texasholdem_type_3.txt
Saved to texasholdem_type_3.txt
Saved to texasholdem_type_3.txt
Saved to texasholdem_type_3.txt
recv: h\/3c5dQs\/Qc\/Ah:1000|-1000:AIPlayer|Human","type":3,"uuid":"ba78b4a1-5733-4f3a-b3e4-e693edbbb975"}
{"action":1,"data":"STATE:1:r200c\/cr300c\/r500r700c\/cr800c:5h6h|2dAs\/6sKhKd\/3h\/4s:800|-800:Human|AIPlayer","type":3,"uuid":"7cd516cc-94dd-45a2-9d8e-5ccfd91e2e75"}
{"action":1,"data":"STATE:2:r300r500r700cr1100r1500c\/r1600c\/r1700r1900cc\/r2000c:7cJd|8cQc\/2hTh4h\/Ah\/Kc:-2000|2000:AIPlayer|Human","type":3,"uuid":"da3e09b3-e28b-4127-b67b-5b73346c27b1"}
{"action":1,"data":"STATE:3:r200r1000cc\/cr1100c\/r1200r1300c\/r2100r2900r4500cc:7dJc|4cQs\/Qd2s5d\/Ad\/Kc:-4500|4500:Human|AIPlayer","type":3,"uuid":"c5366864-f46f-4ac1-be5c-4c9d0e46390b"}
{"action":1,"data":"STATE:4:r200r300r500cr900r1300r2100cc\/r2200c\/r2300c\/r2400c:Jc4c|3s2h\/2d3d5s\/Td\/4h:-2400|2400:AIPlayer|Human","type":3,"uuid":"53dabdb7-ce81-478d-b472-6e560c1bfb51"}
{"action":1,"data":"STAT